Import statements. 

In [1]:
import pandas as pd
import numpy as np
import requests

import geocoder

Use requests and Pandas to obtain the html table of the Wikipedia data. pd.read_html returns a list of DataFrame objects. Only the first entry in this list is important, so it is saved in another variable table.

In [2]:
url='http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
htmlPage = pd.read_html(url)
table = htmlPage[0]
table = table[table['Borough'] != 'Not assigned']

Resetting the index and removing the column created from the old index that was reset.

In [3]:
table.reset_index(inplace = True)
table = table.drop('index', axis = 1)
table.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


This code attempts to request latitude and longitude information for each postal code using geocoder and store that information in two lists. These two lists are used in the next cell to create a dataframe which will be combined with our existing table. If it is not possible to retrieve the geocoder information, the loops break.

In [4]:
# initialize your variable to None
lat_lng_coords = None

latitude = []
longitude = []
# loop until you get the coordinates
for postalCode in table['Postal Code'].values.tolist():
    it = 0
    noRet = 0
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postalCode))
        lat_lng_coords = g.latlng
        it +=1
        print('for PC:',postalCode,'try:', it)
        if it == 75:
            print("Can't retrieve coordinates. Use the provided csv file.")
            noRet = 1
            break
        
    if noRet == 1:
        break
    if it < 75:
        latitude.append(lat_lng_coords[0])
        longitude.append(lat_lng_coords[1])

for PC: M3A try: 1
for PC: M3A try: 2
for PC: M3A try: 3
for PC: M3A try: 4
for PC: M3A try: 5
for PC: M3A try: 6
for PC: M3A try: 7
for PC: M3A try: 8
for PC: M3A try: 9
for PC: M3A try: 10
for PC: M3A try: 11
for PC: M3A try: 12
for PC: M3A try: 13
for PC: M3A try: 14
for PC: M3A try: 15
for PC: M3A try: 16
for PC: M3A try: 17
for PC: M3A try: 18
for PC: M3A try: 19
for PC: M3A try: 20
for PC: M3A try: 21
for PC: M3A try: 22
for PC: M3A try: 23
for PC: M3A try: 24
for PC: M3A try: 25
for PC: M3A try: 26
for PC: M3A try: 27
for PC: M3A try: 28
for PC: M3A try: 29
for PC: M3A try: 30
for PC: M3A try: 31
for PC: M3A try: 32
for PC: M3A try: 33
for PC: M3A try: 34
for PC: M3A try: 35
for PC: M3A try: 36
for PC: M3A try: 37
for PC: M3A try: 38
for PC: M3A try: 39
for PC: M3A try: 40
for PC: M3A try: 41
for PC: M3A try: 42
for PC: M3A try: 43
for PC: M3A try: 44
for PC: M3A try: 45
for PC: M3A try: 46
for PC: M3A try: 47
for PC: M3A try: 48
for PC: M3A try: 49
for PC: M3A try: 50
for PC: M

If the information was able to be obtained from geocoder, the else statement will be called and the dataframe coor

In [7]:
if it >=75:
    coorData = pd.read_csv('https://cocl.us/Geospatial_data')
else:
    data = {'Postal Code':table['Postal Code'].values.tolist(), 'Latitude': latitude, 'Longitude': longitude}
    coorData = pd.DataFrame(data = data, columns = ['Postal Code', 'Latitude', 'Longitude'])
table = table.set_index('Postal Code').join(coorData.set_index('Postal Code')).reset_index()

In [8]:
table.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


AttributeError: module 'geocoder' has no attribute 'register_google'